### **Imports**

In [1]:
from file_io import load as load_image
from joblib import load
import numpy as np
import pandas as pd

### **Auxiliary Function**

_ Inputs_

* __pred -__ the  binary segmentation;
* __gt -__ the ground truth; 
* __mask -__ the binary mask; 
* __ prob -__ the probability map.

_ Outputs _

* __sen -__ Sensibility;
* __spec -__ Specificity; 
* __acc -__ Accuracy; 
* __ auc -__ Area Under the ROC curve;
* __ mcc -__ Matthews Correlation Coefficient.

In [2]:
from sklearn.metrics import confusion_matrix, roc_auc_score, matthews_corrcoef

def perform_metrics (pred, gt, mask, prob):
    
    # Suppressing background regions.
    pred = pred[mask>0]
    gt = gt[mask>0]
    prob = prob[mask>0]

    # Building confusion matrix.
    c_matrix = confusion_matrix(gt, pred)
    
    # Calculating ratios.
    tn = c_matrix[0,0]
    tp = c_matrix[1,1]
    fn = c_matrix[1,0]
    fp = c_matrix[0,1]
    
    # Finding the metrics.
    sen  = tp/(tp+fn)
    spec = tn/(tn+fp)
    acc  = (tp+tn)/(tp+tn+fp+fn)
    auc  = roc_auc_score (gt, prob)
    mcc  = matthews_corrcoef (gt, pred)
    
    
    return sen, spec, acc, auc, mcc

### **Loading**

In [3]:
test_images = [1, 2, 3, 4, 5, 44, 77, 81, 82, 139, 162, 163, 235, 236, 239, 240, 255, 291, 319, 324]
r_width, r_height = 700, 605

In [4]:
gts = np.zeros((len(test_images), r_height, r_width))
masks = np.zeros ((len(test_images), r_height, r_width))
segs = np.zeros((len(test_images), r_height, r_width))
probs = np.zeros((len(test_images), r_height, r_width))

In order for you to run the cell below, please download the annotations from [STARE's website](http://cecas.clemson.edu/~ahoover/stare/) and change the paths accordingly.

In [5]:
#Change these paths
gts_path = '../resources/gts/STARE/'

for i, image in enumerate(test_images):
    gt = load_image(gts_path + 'im%04d.ah.ppm.gz' %image, normalize = True)
    mask = load_image('../resources/masks/STARE/mask_%04d.png' %image)
    seg = load_image('../resources/binary_segmentations/STARE/seg_%04d.png' %image, normalize = True)
    prob = load('../resources/probability_maps/STARE/prob_%04d.npy' %image)[1]
    
    gts[i] = gt
    masks [i] = mask
    segs[i] = seg
    probs[i] = prob

### **Calculating**

In [6]:
sen_list, spec_list, acc_list, auc_list, mcc_list = [], [], [], [], []

for i in range (len(gts)):
    sen, spec, acc, auc, mcc = perform_metrics (segs[i], gts[i], masks[i], probs[i])
    sen_list.append(sen)
    spec_list.append(spec)
    acc_list.append(acc)
    auc_list.append(auc)
    mcc_list.append(mcc)

### __Storing the data__

In [7]:
data = {'Image' : test_images, 'Sensitivity': sen_list, 'Specificity': spec_list, 'Accuracy': acc_list, 
        'AUC': auc_list, 'MCC': mcc_list}
df = pd.DataFrame(data = data)
df = df[['Image', 'Sensitivity','Specificity','Accuracy', 'AUC', 'MCC']]
df.set_index('Image', inplace = True)

### **Image-level Results (all images)**

In [8]:
df

,Sensitivity,Specificity,Accuracy,AUC,MCC
Image,,,,,
1,0.749277,0.984616,0.958727,0.978728,0.779073
2,0.764687,0.990030,0.969156,0.986414,0.807071
3,0.848127,0.984284,0.973069,0.991206,0.823781
4,0.618812,0.990561,0.952169,0.983366,0.715714
5,0.723102,0.982671,0.950291,0.979113,0.759558
44,0.864852,0.988005,0.977068,0.994379,0.857550
77,0.934807,0.977548,0.972842,0.994982,0.869819
81,0.915981,0.983663,0.976719,0.995150,0.877233
82,0.888098,0.988891,0.977997,0.995226,0.884916


### **Average Results (all images)**

In [9]:
df.mean()

Sensitivity    0.831514
Specificity    0.985813
Accuracy       0.969382
AUC            0.990519
MCC            0.833033
dtype: float64

### **Image-level Results (pathological images)**

In [10]:
pathological_images = [1, 2, 3, 4, 5, 44, 139, 291, 319, 324]
df_pathological = df.loc[pathological_images,:]

In [11]:
df_pathological

,Sensitivity,Specificity,Accuracy,AUC,MCC
Image,,,,,
1,0.749277,0.984616,0.958727,0.978728,0.779073
2,0.764687,0.990030,0.969156,0.986414,0.807071
3,0.848127,0.984284,0.973069,0.991206,0.823781
4,0.618812,0.990561,0.952169,0.983366,0.715714
5,0.723102,0.982671,0.950291,0.979113,0.759558
44,0.864852,0.988005,0.977068,0.994379,0.857550
139,0.893774,0.974099,0.965168,0.990420,0.832439
291,0.852669,0.993614,0.983660,0.996362,0.872310
319,0.838423,0.984812,0.976128,0.990317,0.794418


### **Average Results (pathogenic images)**

In [12]:
df_pathological.mean()

Sensitivity    0.798979
Specificity    0.985430
Accuracy       0.967357
AUC            0.987996
MCC            0.805373
dtype: float64